<a href="https://colab.research.google.com/github/NatDar/-Python/blob/main/DNA_NLP_hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install matplotlib-venn

In [3]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [ ]:
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

In [5]:
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [6]:
!pip install cartopy
import cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 30.6 MB/s eta 0:00:00


In [8]:
!pip install stop-words

  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32898 sha256=e5669495c0eae7ee1e943d3e6bc3765f7f2aeb848a8a30db71fe85f123907bb6
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words


In [10]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 64.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=ef5b17c0fec6fd455d475b3eab3f46fdfc0d0fa143ade9a87ecedeb4d770fad7
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [50]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("sample_data/train.csv")
df_test = pd.read_csv("sample_data/test.csv")
df_val = pd.read_csv("sample_data/val.csv")


In [51]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [52]:
print(df_train.columns)
print(df_val.columns)
print(df_test.columns)


Index(['id', 'text', 'class'], dtype='object')
Index(['id', 'text', 'class'], dtype='object')
Index(['id', 'text'], dtype='object')


In [53]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [55]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [56]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [57]:
tokenizer = Tokenizer(num_words=None,
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [58]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [59]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [60]:
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 56s 168ms/step - loss: 0.5585 - accuracy: 0.7016 - val_loss: 0.4949 - val_accuracy: 0.7576
Epoch 2/10
319/319 [==============================] - 54s 170ms/step - loss: 0.2876 - accuracy: 0.8834 - val_loss: 0.5609 - val_accuracy: 0.7457


In [61]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 20ms/step - loss: 0.5930 - accuracy: 0.7294


Test score: 0.5930396318435669
Test accuracy: 0.7294449806213379


Какие проблемы у рекурентных сетей?
затухают градиенты
медленно, нужно всегда дойти до конца
Как решить? -> LSTM


https://colah.github.io/posts/2015-08-Understanding-LSTMs/

Давайте, кратко посмотрим как это работает:

In [62]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 113s 337ms/step - loss: 0.5543 - accuracy: 0.7080 - val_loss: 0.4953 - val_accuracy: 0.7546
Epoch 2/10
319/319 [==============================] - 109s 341ms/step - loss: 0.3268 - accuracy: 0.8648 - val_loss: 0.5349 - val_accuracy: 0.7467


In [63]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 3s 73ms/step - loss: 0.5745 - accuracy: 0.7357


Test score: 0.574547529220581
Test accuracy: 0.7357492446899414


Какие проблемы:
вычислительно сложно -> медленнее
на очень длинных последовательностях все равно затухает градиент
Зачем платить больше - уберем некоторые врата (точнее совместим) -> ускоримся, уменьшим число параметров -> GRU

In [64]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 98s 294ms/step - loss: 0.5502 - accuracy: 0.7126 - val_loss: 0.4905 - val_accuracy: 0.7572
Epoch 2/10
319/319 [==============================] - 92s 289ms/step - loss: 0.3153 - accuracy: 0.8692 - val_loss: 0.5428 - val_accuracy: 0.7465


In [65]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 40ms/step - loss: 0.6109 - accuracy: 0.7308


Test score: 0.6108540892601013
Test accuracy: 0.730767548084259


3 подхода:


Как регуляризовать?

дропаут
рекурентный дропаут


In [66]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense

# Пример параметров
vocab_size = 10000
embedding_dim = 50
max_sequence_length = 100
num_classes = 2

# Создание модели
cnn_model = Sequential()

# Добавление слоев
cnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(num_classes, activation='softmax'))

# Компиляция модели
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Вывод структуры модели
cnn_model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 50)           500000    
                                                                 
 conv1d (Conv1D)             (None, 98, 64)            9664      
                                                                 
 max_pooling1d (MaxPooling1  (None, 49, 64)            0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense_6 (Dense)             (None, 128)               401536    
                                                                 
 dense_7 (Dense)             (None, 2)                 258       
                                                      

Embedding сл
Conv1D слой применяет свертку к входным данным.

MaxPooling1D слой используется для уменьшения размерности данных после свертки.

Flatten слой используется для преобразования данных в одномерный вектор перед подачей их в полносвязные слои.

Dense слои представляют собой полносвязные слои с активацией ReLU (Rectified Linear Unit) и softmax для классификации.

In [67]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Пример параметров
vocab_size = 10000
embedding_dim = 50
max_sequence_length = 100
num_classes = 2

# Создание модели LSTM
lstm_model = Sequential()

# Добавление слоев
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
lstm_model.add(LSTM(64))
lstm_model.add(Dense(num_classes, activation='softmax'))

# Компиляция модели
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Вывод структуры модели
lstm_model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 50)           500000    
                                                                 
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 dense_8 (Dense)             (None, 2)                 130       
                                                                 
Total params: 529570 (2.02 MB)
Trainable params: 529570 (2.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [68]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense

# Пример параметров
vocab_size = 10000
embedding_dim = 50
max_sequence_length = 100
num_classes = 2

# Создание модели GRU
gru_model = Sequential()

# Добавление слоев
gru_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
gru_model.add(GRU(64))
gru_model.add(Dense(num_classes, activation='softmax'))

# Компиляция модели
gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Вывод структуры модели
gru_model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 50)           500000    
                                                                 
 gru_1 (GRU)                 (None, 64)                22272     
                                                                 
 dense_9 (Dense)             (None, 2)                 130       
                                                                 
Total params: 522402 (1.99 MB)
Trainable params: 522402 (1.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [70]:
print(df_train.columns)


Index(['id', 'text', 'class'], dtype='object')


In [73]:
labels = to_categorical(df_train['class'], num_classes=num_classes)



In [75]:
# Пример предобработки текста и подготовки данных (предполагается, что у вас есть данные df_train с колонкой 'text')
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(df_train['text'])
sequences = tokenizer.texts_to_sequences(df_train['text'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Преобразование меток в формат one-hot
labels = to_categorical(df_train['class'], num_classes=num_classes)

# Обучение модели LSTM
lstm_model.fit(padded_sequences, labels, epochs=10, validation_split=0.2)

# Обучение модели GRU
gru_model.fit(padded_sequences, labels, epochs=10, validation_split=0.2)




Epoch 1/10
4537/4537 [==============================] - 384s 84ms/step - loss: 0.6932 - accuracy: 0.5054 - val_loss: 0.6931 - val_accuracy: 0.5070
Epoch 2/10
4537/4537 [==============================] - 398s 88ms/step - loss: 0.6931 - accuracy: 0.5064 - val_loss: 0.6931 - val_accuracy: 0.5070
Epoch 3/10
4537/4537 [==============================] - 399s 88ms/step - loss: 0.6931 - accuracy: 0.5058 - val_loss: 0.6931 - val_accuracy: 0.5070
Epoch 4/10
4537/4537 [==============================] - 397s 88ms/step - loss: 0.6931 - accuracy: 0.5060 - val_loss: 0.6931 - val_accuracy: 0.5070
Epoch 5/10
4537/4537 [==============================] - 401s 88ms/step - loss: 0.6931 - accuracy: 0.5059 - val_loss: 0.6931 - val_accuracy: 0.5070
Epoch 6/10
4537/4537 [==============================] - 400s 88ms/step - loss: 0.6931 - accuracy: 0.5058 - val_loss: 0.6931 - val_accuracy: 0.5070
Epoch 7/10
4537/4537 [==============================] - 401s 88ms/step - loss: 0.6931 - accuracy: 0.5063 - val_loss: 0